In [1]:
from __future__ import print_function, division
from gurobipy import Model, GRB, quicksum, LinExpr
import networkx as nx
import pickle
import numpy, random

In [2]:
with open('../MIP/data/networks_prob/graph_spa_500_0.pickle', "rb") as f:
    main_graph = pickle.load(f)
    
samples = []
m = 100
for j in range(m):
    G = nx.DiGraph()
    for u in main_graph.nodes():
        G.add_node(u)
    for u,v in main_graph.edges():
        if main_graph[u][v]['p']> random.random():
            G.add_edge(u, v)
    samples.append(G)

In [3]:
model = Model('IM')
min_value = model.addVar(lb=0.0, ub=1.0, vtype=GRB.CONTINUOUS)

mvars = []
#active nodes
avars = []
#seed nodes
svars = []
var_seed_dict = {}
var_active_dict = {}
var_mean_dict = {}

for j in range(len(main_graph.nodes())):
    s = model.addVar(lb=0.0, ub=1.0, vtype=GRB.BINARY)
    svars.append(s)
    var_seed_dict[j] = s

for sample_index, sample in enumerate(samples):
    for j in range(len(main_graph.nodes())):
        a = model.addVar(lb=0.0, ub=1.0, vtype=GRB.BINARY)
        avars.append(a)
        var_active_dict[(sample_index,j)] = a    
            
mvars.append(avars)
mvars.append(svars)

In [4]:
budget = 25
model.addConstr(quicksum(svars), GRB.LESS_EQUAL, budget)

<gurobi.Constr *Awaiting Model Update*>

In [5]:
obj_expr = quicksum(avars)
model.setObjective(obj_expr, GRB.MAXIMIZE)

In [6]:
for sample_index, sample in enumerate(samples):
    for i in range(len(main_graph.nodes())):
        neighbors = nx.ancestors(sample, i) 
        e = len(neighbors)
        ai = var_active_dict[(sample_index,i)]
        si = var_seed_dict[i]
        if i in  var_mean_dict.keys():
            var_mean_dict[i]+= ai
        else:
            var_mean_dict[i]= ai
        neighbors_active_vars = []
        neighbors_seed_vars = []
        neighbors_active_vars.append(((e+1), ai))
        neighbors_seed_vars.append(si)
        for neighbor in neighbors:
            neighbors_active_vars.append(((e+1), var_active_dict[(sample_index,neighbor)]))
            neighbors_seed_vars.append(var_seed_dict[neighbor])
        seed_neighbors = quicksum(neighbors_seed_vars)
        model.addConstr(ai, GRB.LESS_EQUAL, seed_neighbors)
        model.addConstr(seed_neighbors, GRB.LESS_EQUAL, LinExpr(neighbors_active_vars)) 

In [7]:
try:
    model.optimize()
except e:
    print(e)

Optimize a model with 100001 rows, 50501 columns and 309613 nonzeros
Variable types: 1 continuous, 50500 integer (50500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 2e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 59417 rows and 28892 columns
Presolve time: 0.97s
Presolved: 40584 rows, 21609 columns, 163395 nonzeros
Variable types: 0 continuous, 21609 integer (21609 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Presolved: 40584 rows, 21609 columns, 163395 nonzeros

Concurrent spin time: 0.75s

Solved with primal simplex

Root relaxation: objective 7.886000e+03, 27588 iterations, 1.83 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    7886.0000000 7886.00000  0

In [8]:
for key,value in var_seed_dict.items():
    if(value.x > 0):
        print(key)

12
13
17
18
34
35
39
40
44
163
238
261
263
264
266
271
278
284
287
298
313
317
461
465
481


In [9]:
optimal = (model.Status == GRB.OPTIMAL)
optimal

True

In [10]:
objective = model.ObjVal
objective

7886.0

In [11]:
objective/(500*m)

0.15772